**Loading the Dataset, performing transformations and splitting into Training, Validation and Test Set**

In [ ]:
# Import Statements

from torch.utils import data
import torch
import torchvision as tv
from torchvision import transforms
import torch.utils.data as dt
import datetime
from torch.utils.tensorboard import SummaryWriter
import random
import numpy as np

In [ ]:
# The location of the dataset in the Drive

path = "/content/drive/MyDrive/images_original"

In [ ]:
# Since we need to convert PIL images to Tensors and Resize all the images, we require 2 transformations
# For this purpose, we can use "transforms.Compose", which takes a list of transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((180, 180))])
dataset = tv.datasets.ImageFolder(path, transform)

In [ ]:
# Splitting the Dataset into Training Set, Validation Set and Test Set
# This can be done using torch.utils.data.random_split() method

train_len = round((70/100) * len(dataset))
val_len = round((20/100) * len(dataset))
test_len = round((10/100) * len(dataset))

# The Generator is set in order to reproduce the results

train_data, val_data, test_data = dt.random_split(dataset, [train_len, val_len, test_len], generator=torch.Generator().manual_seed(42))

In [ ]:
# We use the DataLoader to convert the training set into batches of images
# The batch size is specified as 32
# 2 processors have been used to batch the dataset in parallel
# The training set will be shuffled for each epoch

train_loader = dt.DataLoader(train_data, batch_size = 32, num_workers = 2, shuffle = True)
val_loader = dt.DataLoader(val_data, batch_size = 32, num_workers = 2, shuffle = False)
test_loader = dt.DataLoader(test_data, batch_size = 32, num_workers = 2, shuffle = False)

In [ ]:
# Checking the size of the images to ensure that the Resize Transformation worked

itr = iter(train_loader)
img, label = next(itr)
print(img.size())
print(label.size())
print("Unique labels values = ", label.unique())

In [ ]:
# Setting the Seed to ensure reproducibility

seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

**Training Loop for Network 1 and Network 2:**

In [ ]:
# Training Loop for Network 1 and 2

def train1(epoch, train_loader, model, loss_function, optimiser):

  # To visualise the loss and the accuracy for the training and validation set, TensorBoard is used
  current_time = str(datetime.datetime.now().timestamp())
  log_dir = 'logs/tensorboard/' + current_time

  # The SummaryWriter Class is used to keep a log of the models created for the purpose of visualisation
  writer = SummaryWriter(log_dir)

  for epochs in range(epoch):
    epoch_loss = 0.0

    for img, label in train_loader:

      # Training the model
      img = img.cuda()
      label = label.cuda()
      output = model(img)

      # Finding out how well the model performed using the Loss Function
      # We will use Cross Entropy loss as we dealing with multiple classes
      loss = loss_function(output, label)

      # Finding the best parameters to minimise the Loss function
      # We will be using the Adam optimiser and Backpropagation
      # Adam will move towards the Global Minimum
      # Backpropagation will update the weights

      optimiser.zero_grad()
      loss.backward()
      optimiser.step()

      # Computing the Average loss at each epoch
      # This will give us a smoother curve for visualisation
      epoch_loss = 0.9 * epoch_loss + 0.1 * loss.item()

    print("Traning Loss at Epoch {}: {:.4f}".format(epochs, epoch_loss))

    # Traning and Validation accuracy for each epoch
    train_accuracy = evaluation_function1(train_loader, model)
    validation_accuracy = evaluation_function1(val_loader, model)

    # Giving the Training and Validation Accuracy to the SummaryWriter class to visualise in TensorBoard
    writer.add_scalar("Loss over Epoch", epoch_loss, epochs)
    writer.add_scalars("Accuracy", {"Training Accuracy": train_accuracy, "Validation Accuracy": validation_accuracy}, epochs)

  writer.close()
  return model

**Evaluation Function for Network 1 and 2:**

In [ ]:
# This evaluation function is used for Networks 1 and 2
# Evaluating the perfromance of the network on the Validation set

def evaluation_function1(loader, model):
  total = 0
  for img, label in loader:

    # The data (image, label) are unseen by the model and so is again passed to the GPU
    # We do not pass the model however again
    img = img.cuda()
    label = label.cuda()

    # Each image will have 10 probabilities (1 for each class from index 0 - 9)
    output = model(img)

    # Each index will indicate the class 0 - 9, which will be compared with the label
    _, prediction = torch.max(output, dim = 1)
    total += (prediction == label).sum()

  return (total/len(loader.dataset)).item()

# **Network 1:**

In [ ]:
# Building the Network with 2 Hidden Layers

input_size = 3 * 180 * 180
hidden_size1 = 512
hidden_size2 = 512
output_size = 10

class Net1(torch.nn.Module):

  def __init__(self, in_features, out_features) -> None:
    super().__init__()

    # Input Image size is (32, 3, 180, 180)
    self.hidden_layer1 = torch.nn.Linear(in_features, hidden_size1)

    # Input Image size is (32, 3, 512, 512)
    self.hidden_layer2 = torch.nn.Linear(hidden_size1, hidden_size2)

    # Input Image size is (32, 3, 512, 512)
    self.output_layer = torch.nn.Linear(hidden_size2, out_features)

    # ReLu Layer (non-linear activation function)
    self.relu_layer = torch.nn.ReLU()

  def forward(self, image):

    # Input Layer
    image = image.flatten(start_dim = 1)

    # Hidden Layer 1
    image = self.hidden_layer1(image)
    image = self.relu_layer(image)

    # Hidden Layer 2
    image = self.hidden_layer2(image)
    image = self.relu_layer(image)

    # Output Layer
    image = self.output_layer(image)

    return image

In [ ]:
model1 = Net1(3 * 180 * 180, 10)
model1 = model1.cuda()
loss_function = torch.nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model1.parameters())

In [ ]:
# Loading TensorBoard on the notebook
%load_ext tensorboard

In [ ]:
# Training the model on 50 epochs
!rm -rf logs
model1 = train1(50, train_loader, model1, loss_function, optimiser)
%tensorboard --logdir logs/tensorboard

In [ ]:
# Testing the model run on 50 Epochs
test_accuracy1 = evaluation_function1(test_loader, model1)
print(test_accuracy1)

In [ ]:
# Training the model on 100 epochs
!rm -rf logs
model1 = train1(100, train_loader, model1, loss_function, optimiser)
%tensorboard --logdir logs/tensorboard

In [ ]:
# Testing the model run on 100 Epochs
test_accuracy1 = evaluation_function1(test_loader, model1)
print(test_accuracy1)

# **Network 2:**

In [ ]:
# Building the Convolutional Neural Network

class Net2(torch.nn.Module):
  def __init__(self):
    super().__init__()

    # Input Image size is (32, 3, 180, 180)
    self.conv_layer1 = torch.nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3)

    # Input Image size is (32, 32, 178, 178)
    self.conv_layer2 = torch.nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3)

    # Input Image size is (32, 32, 176, 176)
    self.max_pool_layer1 = torch.nn.MaxPool2d(kernel_size = (2,2))

    # Input Image size is (32, 32, 88, 88)
    self.conv_layer3 = torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)

    # Input Image size is (32, 64, 86, 86)
    self.conv_layer4 = torch.nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)

    # Input Image size is (32, 64, 84, 84)
    self.max_pool_layer2 = torch.nn.MaxPool2d(kernel_size = (2,2))

    # Input Image size is (32, 64, 42, 42)
    self.flatten = torch.nn.Flatten()

    # Input Image size is (32, 64 * 42 * 42)
    self.fc_layer1 = torch.nn.Linear(in_features = 64 * 42 * 42, out_features = 256)
    self.fc_layer2 = torch.nn.Linear(in_features = 256, out_features = 10)

    # ReLu Layer (non-linear activation function)
    self.relu_layer = torch.nn.ReLU()

  def forward(self, image):

    # 2 Convolution Layers followed by ReLU (Non-linear Activation Function)
    image = self.conv_layer1(image)
    image = self.relu_layer(image)
    image = self.conv_layer2(image)
    image = self.relu_layer(image)
    # Max Pooling Layer 1
    image = self.max_pool_layer1(image)

    # 2 Convolution Layers followed by ReLU (Non-linear Activation Function)
    image = self.conv_layer3(image)
    image = self.relu_layer(image)
    image = self.conv_layer4(image)
    image = self.relu_layer(image)
    # Max Pooling Layer 2
    image = self.max_pool_layer1(image)

    # Flattening the image before passing it to the Fully Connected Network
    image = self.flatten(image)

    # Fully Connected Layer 1
    image = self.fc_layer1(image)
    image = self.relu_layer(image)

    # Fully Connected Layer 2 (Predicts the output)
    image = self.fc_layer2(image)

    return image

In [ ]:
model2 = Net2()
model2 = model2.cuda()
loss_function = torch.nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model2.parameters())

In [ ]:
# Training the model on 50 epochs
!rm -rf logs
model2 = train1(50, train_loader, model2, loss_function, optimiser)
%tensorboard --logdir logs/tensorboard

In [ ]:
# Testing the model run on 50 Epochs
test_accuracy2 = evaluation_function1(test_loader, model2)
print(test_accuracy2)

In [ ]:
# Training the model on 100 epochs
!rm -rf logs
model2 = train1(100, train_loader, model2, loss_function, optimiser)
%tensorboard --logdir logs/tensorboard

In [ ]:
# Testing the model run on 100 Epochs
test_accuracy2 = evaluation_function1(test_loader, model2)
print(test_accuracy2)

**Training Loop for Network 3 and 4:**

In [ ]:
# Training Loop for Network 3 and 4

def train2(epoch, train_loader, model, loss_function, optimiser):

  # To visualise the loss and the accuracy for the training and validation set, TensorBoard is used
  current_time = str(datetime.datetime.now().timestamp())
  log_dir = 'logs/tensorboard/' + current_time

  # The SummaryWriter Class is used to keep a log of the models created for the purpose of visualisation
  writer = SummaryWriter(log_dir)

  for epochs in range(epoch):
    epoch_loss = 0.0

    # This command is called to allow the model to compute the running mean and average
    # This is a book keeping step and does not have any effect on the training
    model.train()

    for img, label in train_loader:

      # Training the model
      img = img.cuda()
      label = label.cuda()
      output = model(img)

      # Finding out how well the model performed using the Loss Function
      # We will use Cross Entropy loss as we dealing with multiple classes
      loss = loss_function(output, label)

      # Finding the best parameters to minimise the Loss function
      # We will be using Adam/RMSprop and Backpropagation
      # Adam/RMSprop will move towards the Global Minimum
      # Backpropagation will update the weights
      optimiser.zero_grad()
      loss.backward()
      optimiser.step()

      # Computing the Average loss at each epoch
      # This will give us a smoother curve for visualisation
      epoch_loss = 0.9 * epoch_loss + 0.1 * loss.item()

    print("Traning Loss at Epoch {}: {:.4f}".format(epochs, epoch_loss))

    # Training and Validation accuracy for each epoch
    train_accuracy = evaluation_function2(train_loader, model)
    validation_accuracy = evaluation_function2(val_loader, model)

    # Giving the Training and Validation Accuracy to the SummaryWriter class to visualise in TensorBoard
    writer.add_scalar("Loss over Epoch", epoch_loss, epochs)
    writer.add_scalars("Accuracy", {"Training Accuracy": train_accuracy, "Validation Accuracy": validation_accuracy}, epochs)

  writer.close()
  return model

**Evaluation function for Network 3 and Network 4:**

In [ ]:
# Evaluation function for network 3 and 4

def evaluate(batch, model):
  total = 0
  img, label = batch

  # The data (image, label) are unseen by the model and so is again passed to the GPU
  # We do not pass the model however again
  img = img.cuda()
  label = label.cuda()

  # Each image will have 10 probabilities (1 for each class from index 0 - 9)
  output = model(img)

  # Each index will indicate the class 0 - 9, which will be compared with the label
  _, prediction = torch.max(output, dim = 1)
  total += (prediction == label).sum()

  return (total/len(label)).item()

@torch.no_grad()
def evaluation_function2(loader, model):

  # This command prevents running averages over the mean and variance
  # It is used to evaluate the model
  model.eval()

  l_accuracy = []
  for batch in loader:
    l_accuracy.append(evaluate(batch, model))
  l_accuracy = torch.tensor(l_accuracy)
  accuracy = l_accuracy.mean()
  return accuracy

# **Network 3:**

In [ ]:
# Building the Convolutional Neural Network

class Net3(torch.nn.Module):
  def __init__(self):
    super().__init__()

    # Input Image size is (32, 3, 180, 180)
    self.conv_layer1 = torch.nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3)

    # Input Image size is (32, 32, 178, 178)
    self.conv_layer2 = torch.nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3)

    # Input Image size is (32, 32, 176, 176)
    self.max_pool_layer1 = torch.nn.MaxPool2d(kernel_size = (2,2))

    # Input Image size is (32, 32, 88, 88)
    self.conv_layer3 = torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)

    # Input Image size is (32, 64, 86, 86)
    self.conv_layer4 = torch.nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)

    # Input Image size is (32, 64, 84, 84)
    self.max_pool_layer2 = torch.nn.MaxPool2d(kernel_size = (2,2))

    # Input Image size is (32, 64, 42, 42)
    self.flatten = torch.nn.Flatten()

    # Input Image size is (32, 64 * 42 * 42)
    self.fc_layer1 = torch.nn.Linear(in_features = 64 * 42 * 42, out_features = 256)
    self.fc_layer2 = torch.nn.Linear(in_features = 256, out_features = 10)

    # ReLu Layer (non-linear activation function)
    self.relu_layer = torch.nn.ReLU()

    # Batch Normalisation Layer
    # The parameter num_features represents the number of features channels from the previous Convolution Layer
    self.batch_norm1 = torch.nn.BatchNorm2d(num_features = 32)
    self.batch_norm2 = torch.nn.BatchNorm2d(num_features = 32)
    self.batch_norm3 = torch.nn.BatchNorm2d(num_features = 64)
    self.batch_norm4 = torch.nn.BatchNorm2d(num_features = 64)

  def forward(self, image):

    # 2 Convolution Layers and Batch Normalisation Layers followed by ReLU (Non-linear Activation Function)
    image = self.conv_layer1(image)
    image = self.batch_norm1(image)
    image = self.relu_layer(image)
    image = self.conv_layer2(image)
    image = self.batch_norm2(image)
    image = self.relu_layer(image)
    # Max Pooling Layer 1
    image = self.max_pool_layer1(image)

    # 2 Convolution Layers and Batch Normalisation Layers followed by ReLU (Non-linear Activation Function)
    image = self.conv_layer3(image)
    image = self.batch_norm3(image)
    image = self.relu_layer(image)
    image = self.conv_layer4(image)
    image = self.batch_norm4(image)
    image = self.relu_layer(image)
    # Max Pooling Layer 2
    image = self.max_pool_layer1(image)

    # Flattening the image before passing it to the Fully Connected Network
    image = self.flatten(image)

    # Fully Connected Layer 1
    image = self.fc_layer1(image)
    image = self.relu_layer(image)

    # Fully Connected Layer 2 (Predicts the output)
    image = self.fc_layer2(image)

    return image

In [ ]:
model3 = Net3()
model3 = model3.cuda()
loss_function = torch.nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model3.parameters())

In [ ]:
# Training the model on 50 epochs
!rm -rf logs
model3 = train2(50, train_loader, model3, loss_function, optimiser)
%tensorboard --logdir logs/tensorboard

In [ ]:
# Testing the model run on 50 Epochs
test_accuracy3 = evaluation_function2(test_loader, model3)
print(test_accuracy3)

In [ ]:
# Training the model on 100 epochs
!rm -rf logs
model3 = train2(100, train_loader, model3, loss_function, optimiser)
%tensorboard --logdir logs/tensorboard

In [ ]:
# Testing the model run on 100 Epochs
test_accuracy3 = evaluation_function2(test_loader, model3)
print(test_accuracy3)

# **Network 4:**
This network has the same architecture as Network 3, but uses the RMSProp Optimiser.

In [ ]:
model4 = Net3()
model4 = model4.cuda()
loss_function = torch.nn.CrossEntropyLoss()
optimiser = torch.optim.RMSprop(model4.parameters(), lr = 0.001, momentum = 0.9)

In [ ]:
# Training the model on 50 epochs
!rm -rf logs
model4 = train2(50, train_loader, model4, loss_function, optimiser)
%tensorboard --logdir logs/tensorboard

In [ ]:
# Testing the model run on 50 Epochs
test_accuracy4 = evaluation_function2(test_loader, model4)
print(test_accuracy4)

In [ ]:
# Training the model on 100 epochs
!rm -rf logs
model4 = train2(100, train_loader, model4, loss_function, optimiser)
%tensorboard --logdir logs/tensorboard

In [ ]:
# Testing the model run on 100 Epochs
test_accuracy4 = evaluation_function2(test_loader, model4)
print(test_accuracy4)